In [1]:
from utils import read_video, save_video
from trackers import Tracker
import cv2
from team_assigner import TeamAssigner

In [2]:
# Read Video
video_path = 'input_video/08fd33_4.mp4'
output_dir = 'output_video'

In [3]:
video_frames = read_video(video_path)

In [4]:
# 初始化Tracker
# tracker = Tracker('models/best_yolov5_100.pt')
tracker = Tracker('models/train3_5xu/best.pt')
tracks = tracker.get_object_tracks(video_frames,
                                    read_from_stub=True,
                                    stub_path='stub/track_stubs.pkl')

In [5]:
# 对球插值
tracks['ball'] = tracker.interpolate_ball(tracks['ball'])

In [7]:
# 保存每个队员的截图, 只保存第一帧的，只一次
for id, player in tracks["players"][0].items():
    bbox = player["bbox"]
    frame = video_frames[0]
    cropped = frame[int(bbox[1]) : int(bbox[3]), int(bbox[0]) : int(bbox[2])]
    cv2.imwrite(f"{output_dir}/player_{id}.jpg", cropped)

In [8]:
# 分配队员
team_assigner = TeamAssigner()
team_assigner.assign_team(video_frames[0], 
                          tracks['players'][0])
for frame_num, player_track in enumerate(tracks['players']):
    for player_id, track in player_track.items():
        team = team_assigner.get_player_team(video_frames[frame_num], 
                                             track['bbox'],
                                                player_id)
        tracks['players'][frame_num][player_id]['team'] = team
        tracks['players'][frame_num][player_id]['team_color'] = team_assigner.team_colors[team]


In [9]:
# 画出输出
## 画出对象轨迹
output_video_frames = tracker.draw_annotation(video_frames, tracks)

In [10]:
# Save Video
save_video(output_video_frames, f'{output_dir}/output_video.avi')

Saving Video: 100%|██████████| 750/750 [00:10<00:00, 71.88it/s]

Video saved at output_video/output_video.avi
